In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Baseline Model

In [2]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import f1_score
# import numpy as np

# # Load data
# train_df = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/test.csv')
# test_df = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/test.csv')

# emotion_cols = ['anger', 'fear', 'joy', 'sadness', 'surprise']

# # Feature extraction
# vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
# X_train = vectorizer.fit_transform(train_df['text'])
# X_test = vectorizer.transform(test_df['text'])

# y_train = train_df[emotion_cols].values

# # Train model
# model = MultiOutputClassifier(LogisticRegression(max_iter=1000, C=1.0))
# model.fit(X_train, y_train)

# # Predict
# predictions = model.predict(X_test)

# # Create submission
# submission = pd.DataFrame({
#     'id': test_df['id'],
#     'anger': predictions[:, 0],
#     'fear': predictions[:, 1],
#     'joy': predictions[:, 2],
#     'sadness': predictions[:, 3],
#     'surprise': predictions[:, 4]
# })

# submission.to_csv('submission.csv', index=False)
# print(submission.head())


## Multi-Label Emotion Classification - Milestone 1


### ============================================================================
## CELL 1: Import Libraries and Load Data
### ============================================================================

In [3]:

import pandas as pd
import numpy as np
import string
import nltk
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Download NLTK resources (run once)
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Load data
train_df = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/train.csv')
test_df = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/test.csv')

print("✅ Data loaded successfully!")
print(f"Training samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")
print("\nFirst 3 rows:")
print(train_df.head(3))


✅ Data loaded successfully!
Training samples: 6827
Test samples: 1707

First 3 rows:
   id                                               text  anger  fear  joy  \
0   0  the dentist that did the work apparently did a...      1     0    0   
1   1  i'm gonna absolutely ~~suck~~ be terrible duri...      0     1    0   
2   2  bridge: so leave me drowning calling houston, ...      0     1    0   

   sadness  surprise             emotions  
0        1         0  ['anger' 'sadness']  
1        1         0   ['fear' 'sadness']  
2        1         0   ['fear' 'sadness']  


### ============================================================================
## CELL 2: Milestone Questions 1-7 - Label Analysis
### ============================================================================

In [4]:
print("="*80)
print("MILESTONE QUESTIONS 1-7: LABEL ANALYSIS")
print("="*80)

emotion_cols = ['anger', 'fear', 'joy', 'sadness', 'surprise']

# -------------------------------------------------------------------------
# Q1: Most common emotion
# -------------------------------------------------------------------------
emotion_counts = train_df[emotion_cols].sum()
most_common_emotion = emotion_counts.idxmax()
most_common_count = emotion_counts[most_common_emotion]
least_common_emotion = emotion_counts.idxmin()

print(f"\n📊 Q1: Most common emotion")
print(emotion_counts.sort_values(ascending=False))
print(f"✅ ANSWER: {most_common_emotion} ({most_common_count} occurrences)")

# -------------------------------------------------------------------------
# Q2: Instances with exactly 2 labels
# -------------------------------------------------------------------------
train_df['label_count'] = train_df[emotion_cols].sum(axis=1)
exactly_2_labels = (train_df['label_count'] == 2).sum()

print(f"\n📊 Q2: Instances with exactly 2 labels")
print(f"✅ ANSWER: {exactly_2_labels}")
print("\nLabel distribution:")
print(train_df['label_count'].value_counts().sort_index())

# -------------------------------------------------------------------------
# Q3: Joy and Sadness together
# -------------------------------------------------------------------------
joy_and_sadness = ((train_df['joy'] == 1) & (train_df['sadness'] == 1)).sum()
print(f"\n📊 Q3: Joy and Sadness together")
print(f"✅ ANSWER: {joy_and_sadness}")

# -------------------------------------------------------------------------
# Q4: Percentage containing Surprise
# -------------------------------------------------------------------------
surprise_count = train_df['surprise'].sum()
surprise_percentage = (surprise_count / len(train_df)) * 100
print(f"\n📊 Q4: Percentage with Surprise")
print(f"✅ ANSWER: {surprise_percentage:.2f}%")

# -------------------------------------------------------------------------
# Q5: Maximum difference
# -------------------------------------------------------------------------
max_diff = emotion_counts.max() - emotion_counts.min()
print(f"\n📊 Q5: Max difference in occurrence counts")
print(f"Max: {emotion_counts.idxmax()} ({emotion_counts.max()})")
print(f"Min: {emotion_counts.idxmin()} ({emotion_counts.min()})")
print(f"✅ ANSWER: {max_diff}")

# -------------------------------------------------------------------------
# Q6: Median word length
# -------------------------------------------------------------------------
train_df['word_count'] = train_df['text'].apply(lambda x: len(str(x).split()))
median_word_count = train_df['word_count'].median()
print(f"\n📊 Q6: Median word length")
print(f"✅ ANSWER: {median_word_count}")

# -------------------------------------------------------------------------
# Q7: Correlation between anger and fear
# -------------------------------------------------------------------------
correlation = train_df['anger'].corr(train_df['fear'])
print(f"\n📊 Q7: Correlation (anger & fear)")
print(f"✅ ANSWER: {correlation:.2f}")

# Full correlation matrix
print("\nFull correlation matrix:")
corr_matrix = train_df[emotion_cols].corr()
print(corr_matrix.round(2))


MILESTONE QUESTIONS 1-7: LABEL ANALYSIS

📊 Q1: Most common emotion
fear        3860
sadness     2171
surprise    1999
joy         1660
anger        808
dtype: int64
✅ ANSWER: fear (3860 occurrences)

📊 Q2: Instances with exactly 2 labels
✅ ANSWER: 2587

Label distribution:
label_count
0     676
1    2743
2    2587
3     706
4     112
5       3
Name: count, dtype: int64

📊 Q3: Joy and Sadness together
✅ ANSWER: 96

📊 Q4: Percentage with Surprise
✅ ANSWER: 29.28%

📊 Q5: Max difference in occurrence counts
Max: fear (3860)
Min: anger (808)
✅ ANSWER: 3052

📊 Q6: Median word length
✅ ANSWER: 13.0

📊 Q7: Correlation (anger & fear)
✅ ANSWER: 0.08

Full correlation matrix:
          anger  fear   joy  sadness  surprise
anger      1.00  0.08 -0.19     0.09      0.02
fear       0.08  1.00 -0.47     0.29      0.16
joy       -0.19 -0.47  1.00    -0.32     -0.10
sadness    0.09  0.29 -0.32     1.00     -0.12
surprise   0.02  0.16 -0.10    -0.12      1.00


# ============================================================================
# CELL 3: Concept - Text Normalization
# ============================================================================


In [5]:
print("\n" + "="*80)
print("CONCEPT 1: TEXT NORMALIZATION (Lowercasing)")
print("="*80)

# Why normalize?
# "Happy", "happy", and "HAPPY" should be treated as the same word

# Impact on vocabulary
sample_texts = ["Happy birthday!", "I'm happy today", "HAPPY times"]
words_before = set(" ".join(sample_texts).split())
words_after = set(" ".join(sample_texts).lower().split())
print(f"\nVocabulary size before: {len(words_before)}")
print(f"Vocabulary size after: {len(words_after)}")
print(f"Reduction: {len(words_before) - len(words_after)} unique words merged")



CONCEPT 1: TEXT NORMALIZATION (Lowercasing)

Vocabulary size before: 7
Vocabulary size after: 5
Reduction: 2 unique words merged


# ============================================================================
# CELL 4: Concept - Tokenization
# ============================================================================

In [6]:
print("\n" + "="*80)
print("CONCEPT 2: TOKENIZATION")
print("="*80)

# Tokenization = Breaking text into words/tokens
# Why? ML models need individual words as features

sample = "I'm loving this movie! It's amazing, isn't it?"
print(f"Text: {sample}")

# Method 1: Simple split (naive)
tokens_simple = sample.split()
print(f"\nSimple split: {tokens_simple}")
print(f"Count: {len(tokens_simple)}")

# Method 2: NLTK (better - handles contractions, punctuation)
tokens_nltk = word_tokenize(sample)
print(f"\nNLTK tokenize: {tokens_nltk}")
print(f"Count: {len(tokens_nltk)}")

# Key difference: NLTK separates "I'm" → ["I", "'m"], "isn't" → ["is", "n't"]
# This is better for understanding linguistic structure

# Apply to real data
print("\nExample from dataset:")
sample_row = train_df['text'].iloc[0]
print(f"Text: {sample_row[:80]}...")
print(f"Tokens (first 10): {word_tokenize(sample_row.lower())[:10]}")



CONCEPT 2: TOKENIZATION
Text: I'm loving this movie! It's amazing, isn't it?

Simple split: ["I'm", 'loving', 'this', 'movie!', "It's", 'amazing,', "isn't", 'it?']
Count: 8

NLTK tokenize: ['I', "'m", 'loving', 'this', 'movie', '!', 'It', "'s", 'amazing', ',', 'is', "n't", 'it', '?']
Count: 14

Example from dataset:
Text: the dentist that did the work apparently did a lousy job as in just a few years ...
Tokens (first 10): ['the', 'dentist', 'that', 'did', 'the', 'work', 'apparently', 'did', 'a', 'lousy']


# ============================================================================
# CELL 5: Concept - Punctuation Removal + Q8
# ============================================================================

In [7]:
print("\n" + "="*80)
print("CONCEPT 3: PUNCTUATION REMOVAL")
print("="*80)

# What punctuation are we removing?
print(f"string.punctuation = {string.punctuation}")

# Example
text_punct = "Hello! How are you? I'm fine, thanks."
print(f"\nOriginal: {text_punct}")

# Remove punctuation using translate (fastest method)
text_no_punct = text_punct.translate(str.maketrans('', '', string.punctuation))
print(f"Cleaned: {text_no_punct}")

# Why remove?
# 1. Reduces vocabulary: "word" vs "word!" vs "word?"
# 2. Simplifies processing
# 3. Most punctuation doesn't carry emotion info

# -------------------------------------------------------------------------
# Q8: Character reduction after removing punctuation
# -------------------------------------------------------------------------
print("\n" + "="*80)
print("Q8: CHARACTER REDUCTION AFTER REMOVING PUNCTUATION")
print("="*80)

# Count original characters
original_text = ' '.join(train_df['text'].astype(str))
original_chars = len(original_text)

# Clean function: lowercase + remove punctuation
def clean_text(text):
    text = str(text).lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

train_df['text_clean'] = train_df['text'].apply(clean_text)

# Count cleaned characters
cleaned_text = ' '.join(train_df['text_clean'])
cleaned_chars = len(cleaned_text)

# Calculate reduction
reduction_pct = ((original_chars - cleaned_chars) / original_chars) * 100

print(f"Original character count: {original_chars:,}")
print(f"Cleaned character count: {cleaned_chars:,}")
print(f"Characters removed: {original_chars - cleaned_chars:,}")
print(f"\n✅ ANSWER Q8: {reduction_pct:.2f}% reduction")



CONCEPT 3: PUNCTUATION REMOVAL
string.punctuation = !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

Original: Hello! How are you? I'm fine, thanks.
Cleaned: Hello How are you Im fine thanks

Q8: CHARACTER REDUCTION AFTER REMOVING PUNCTUATION
Original character count: 549,100
Cleaned character count: 531,406
Characters removed: 17,694

✅ ANSWER Q8: 3.22% reduction


# ============================================================================
# CELL 6: Concept - Stop Words
# ============================================================================

In [8]:
# ============================================================================
# CELL 6: Concept - Stop Words + Q9
# ============================================================================
print("\n" + "="*80)
print("CONCEPT 4: STOP WORDS")
print("="*80)

# Stop words = Common words with little semantic value
# Examples: "the", "is", "at", "which", "a", "an"

stop_words = set(stopwords.words('english'))
print(f"Total stop words in NLTK: {len(stop_words)}")
print(f"\nFirst 30 stop words:\n{sorted(list(stop_words))[:30]}")

# Example: Effect of removing stop words
sample = "The quick brown fox jumps over the lazy dog"
tokens = sample.lower().split()
filtered = [w for w in tokens if w not in stop_words]

print(f"\nOriginal: {tokens}")
print(f"After removing stop words: {filtered}")
print(f"Removed: {[w for w in tokens if w in stop_words]}")

# Important note for emotion detection:
# Be careful! "not happy" → "happy" changes meaning completely!
# For this milestone we remove them, but for modeling you might want to keep negations

# -------------------------------------------------------------------------
# Q9: Percentage of unique words that are stop words
# -------------------------------------------------------------------------
print("\n" + "="*80)
print("Q9: PERCENTAGE OF UNIQUE WORDS THAT ARE STOP WORDS")
print("="*80)

# Get all words from cleaned text
all_words = cleaned_text.split()
unique_words = {word for word in set(all_words) if word}  # Remove empty strings

print(f"Total words (with repetition): {len(all_words):,}")
print(f"Unique words: {len(unique_words):,}")

# Find stop words in our vocabulary
unique_stopwords = unique_words.intersection(stop_words)
print(f"Unique stop words found: {len(unique_stopwords)}")

# Calculate percentage
stopword_pct = (len(unique_stopwords) / len(unique_words)) * 100

print(f"\n✅ ANSWER Q9: {stopword_pct:.2f}%")
print(f"\nExamples found: {sorted(list(unique_stopwords))[:20]}")



CONCEPT 4: STOP WORDS
Total stop words in NLTK: 198

First 30 stop words:
['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't"]

Original: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
After removing stop words: ['quick', 'brown', 'fox', 'jumps', 'lazy', 'dog']
Removed: ['the', 'over', 'the']

Q9: PERCENTAGE OF UNIQUE WORDS THAT ARE STOP WORDS
Total words (with repetition): 105,749
Unique words: 8,365
Unique stop words found: 129

✅ ANSWER Q9: 1.54%

Examples found: ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below']


# ============================================================================
# CELL 7: Word Frequency Analysis + Q10
# ============================================================================

In [9]:
print("\n" + "="*80)
print("Q10: 5TH MOST FREQUENT WORD (EXCLUDING STOP WORDS)")
print("="*80)

# Filter out stop words
words_no_stops = [w for w in all_words if w not in stop_words and w != '']
print(f"Total words after removing stop words: {len(words_no_stops):,}")

# Count word frequencies using Counter
word_freq = Counter(words_no_stops)
top_20 = word_freq.most_common(20)

print("\nTop 20 most frequent words (excluding stop words):")
print("-" * 50)
for rank, (word, count) in enumerate(top_20, 1):
    marker = " ⭐ 5TH!" if rank == 5 else ""
    print(f"{rank:2d}. {word:20s} : {count:6,}{marker}")

fifth_word = top_20[4][0]
fifth_count = top_20[4][1]

print(f"\n✅ ANSWER Q10: '{fifth_word}' (appears {fifth_count:,} times)")



Q10: 5TH MOST FREQUENT WORD (EXCLUDING STOP WORDS)
Total words after removing stop words: 52,236

Top 20 most frequent words (excluding stop words):
--------------------------------------------------
 1. head                 :    539
 2. eyes                 :    438
 3. like                 :    394
 4. back                 :    365
 5. heart                :    334 ⭐ 5TH!
 6. one                  :    323
 7. face                 :    293
 8. get                  :    291
 9. time                 :    271
10. still                :    271
11. im                   :    253
12. got                  :    243
13. never                :    220
14. hands                :    217
15. really               :    206
16. hand                 :    198
17. felt                 :    196
18. know                 :    194
19. went                 :    189
20. day                  :    188

✅ ANSWER Q10: 'heart' (appears 334 times)


# ============================================================================
# CELL 9: Concept - Lemmatization
# ============================================================================

In [10]:
# ============================================================================
# CELL 8: Concept - Stemming
# ============================================================================
print("\n" + "="*80)
print("CONCEPT 5: STEMMING")
print("="*80)

# Stemming = Reducing words to their root form (stem)
# Uses rules/heuristics (crude but fast)

stemmer = PorterStemmer()

# Examples
words_to_stem = ['running', 'runs', 'ran', 'runner', 'easily', 'fairly']
print("Stemming examples:")
for word in words_to_stem:
    stemmed = stemmer.stem(word)
    print(f"  {word:15s} → {stemmed}")

# Emotions example
emotion_words = ['loving', 'loved', 'loves', 'lovely', 'happier', 'happiest', 'happiness']
print("\nEmotion words:")
for word in emotion_words:
    stemmed = stemmer.stem(word)
    print(f"  {word:15s} → {stemmed}")

# Apply to sample text
sample_text = "I am feeling extremely happy and loved by my loving family"
tokens = word_tokenize(sample_text.lower())
stemmed_tokens = [stemmer.stem(token) for token in tokens]

print(f"\nOriginal tokens: {tokens}")
print(f"Stemmed tokens:  {stemmed_tokens}")

# Pros: Fast, reduces vocabulary
# Cons: Can be too aggressive ("university" → "univers"), loses meaning



CONCEPT 5: STEMMING
Stemming examples:
  running         → run
  runs            → run
  ran             → ran
  runner          → runner
  easily          → easili
  fairly          → fairli

Emotion words:
  loving          → love
  loved           → love
  loves           → love
  lovely          → love
  happier         → happier
  happiest        → happiest
  happiness       → happi

Original tokens: ['i', 'am', 'feeling', 'extremely', 'happy', 'and', 'loved', 'by', 'my', 'loving', 'family']
Stemmed tokens:  ['i', 'am', 'feel', 'extrem', 'happi', 'and', 'love', 'by', 'my', 'love', 'famili']


In [11]:
# ============================================================================
# CELL 10: Complete Preprocessing Pipeline
# ============================================================================
print("\n" + "="*80)
print("COMPLETE PREPROCESSING PIPELINE")
print("="*80)

def preprocess_text(text, method='lemma', remove_stopwords=False):
    """
    Complete text preprocessing pipeline
    
    Args:
        text: Input text string
        method: 'stem', 'lemma', or 'none'
        remove_stopwords: Boolean to remove stop words
    
    Returns:
        Preprocessed text string
    """
    # 1. Lowercase
    text = str(text).lower()
    
    # 2. Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # 3. Tokenize
    tokens = word_tokenize(text)
    
    # 4. Remove stop words (optional)
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = [t for t in tokens if t not in stop_words]
    
    # 5. Stemming or Lemmatization
    if method == 'stem':
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
    elif method == 'lemma':
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(t, pos='v') for t in tokens]
    
    # 6. Join back to string
    return ' '.join(tokens)

# Test the pipeline
test_texts = [
    "I'm feeling extremely happy and loved!",
    "This is the worst movie I've ever seen.",
    "Surprisingly, the ending was better than expected."
]

print("\nPreprocessing examples:\n")
for text in test_texts:
    print(f"Original: {text}")
    print(f"  Basic:    {preprocess_text(text, method='none')}")
    print(f"  Stemmed:  {preprocess_text(text, method='stem', remove_stopwords=True)}")
    print(f"  Lemma:    {preprocess_text(text, method='lemma', remove_stopwords=True)}")
    print()

# # Apply to dataset (choose your approach)
# print("Applying to training data...")
# train_df['text_processed'] = train_df['text'].apply(
#     lambda x: preprocess_text(x, method='lemma', remove_stopwords=False)
# )

# print("✅ Preprocessing complete!")
# print(f"\nExample comparison:")
# print(f"Original: {train_df['text'].iloc[0][:100]}...")
# print(f"Processed: {train_df['text_processed'].iloc[0][:100]}...")



COMPLETE PREPROCESSING PIPELINE

Preprocessing examples:

Original: I'm feeling extremely happy and loved!
  Basic:    im feeling extremely happy and loved
  Stemmed:  im feel extrem happi love
  Lemma:    im feel extremely happy love

Original: This is the worst movie I've ever seen.
  Basic:    this is the worst movie ive ever seen
  Stemmed:  worst movi ive ever seen
  Lemma:    worst movie ive ever see

Original: Surprisingly, the ending was better than expected.
  Basic:    surprisingly the ending was better than expected
  Stemmed:  surprisingli end better expect
  Lemma:    surprisingly end better expect



# Lemmatization

In [12]:
train_df['text_processed'] = train_df['text'].apply(
    lambda x: preprocess_text(x, method='lemma', remove_stopwords=False)
)

In [13]:
print(f"Original: {train_df['text'].iloc[10:][:100]}...")
print(f"Processed: {train_df['text_processed'].iloc[10:][:100]}...")

Original: 10                               " and burst into tears.
11                         there's nothing but dead air.
12     it's nice to make it from week to week without...
13     the insides of my eyelids flashed bright color...
14     well, now my mouth is just a big ball of ouchies.
                             ...                        
105         but oh god my nails are black call a priest.
106                   so she didn't date my best friend?
107                     a grin slowly formed on my face.
108    he always assumed he was hallucinating or just...
109    most mornings i drift in and out, slowly shaki...
Name: text, Length: 100, dtype: object...
Processed: 10                                   and burst into tear
11                           theres nothing but dead air
12     its nice to make it from week to week without ...
13     the insides of my eyelids flash bright color a...
14       well now my mouth be just a big ball of ouchies
                         

In [14]:
train_df['text_processed'] = train_df['text'].apply(
    lambda x: preprocess_text(x, method='stem', remove_stopwords=False)
)

# Stemming

In [15]:
print(f"Original: {train_df['text'].iloc[10:][:100]}...")
print(f"Processed: {train_df['text_processed'].iloc[10:][:100]}...")

Original: 10                               " and burst into tears.
11                         there's nothing but dead air.
12     it's nice to make it from week to week without...
13     the insides of my eyelids flashed bright color...
14     well, now my mouth is just a big ball of ouchies.
                             ...                        
105         but oh god my nails are black call a priest.
106                   so she didn't date my best friend?
107                     a grin slowly formed on my face.
108    he always assumed he was hallucinating or just...
109    most mornings i drift in and out, slowly shaki...
Name: text, Length: 100, dtype: object...
Processed: 10                                   and burst into tear
11                               there noth but dead air
12     it nice to make it from week to week without a...
13     the insid of my eyelid flash bright color all ...
14         well now my mouth is just a big ball of ouchi
                         